# Testing the HDF5 Library with Jupyter/BeakerX Java kernel

— Agostino De Marco, University of Naples Federico II

This Jupyter notebook uses the [BeakerX Java kernel](http://beakerx.com). The purpose is demonstrating the use of Java and of external native libraries in the context of a notebook. The library used here is [HDF5 by the HDF Group](https://www.hdfgroup.org/), a high performance data software library and file format to manage, process, and store your heterogeneous data. HDF5 is built for fast I/O processing and storage.

## Classpath extras

In [1]:
%classpath add jar ./jarh4obj.jar
%classpath add jar ./jarh5obj.jar
%classpath add jar ./jarhdfobj.jar
%classpath add jar ./jarhdf-2.10.1.jar
%classpath add jar ./jarhdf5-2.10.1.jar
%classpath add jar ./slf4j-api-1.7.5.jar
%classpath

Added jar: [jarh4obj.jar]
Added jar: [jarh5obj.jar]
Added jar: [jarhdfobj.jar]
Added jar: [jarhdf-2.10.1.jar]
Added jar: [jarhdf5-2.10.1.jar]
Added jar: [slf4j-api-1.7.5.jar]
C:\Users\agodemar\AppData\Local\Temp\beaker5688901765675768403\outDir
C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\jarh4obj.jar
C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\jarh5obj.jar
C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\jarhdfobj.jar
C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\jarhdf-2.10.1.jar
C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\jarhdf5-2.10.1.jar
C:\Users\agodemar\Dropbox\Jupyter_2\beakerx\slf4j-api-1.7.5.jar


## Check the native libs

In [2]:
import java.nio.file.*;
Files.list(Paths.get(".")) // a stream of paths
    .map(path -> path.toString().toLowerCase())
    .filter(name -> name.matches("([^\\s]+(\\.(?i)(lib|dll))$)")) // use regex
    .forEach(System.out::println);

.\concrt140.dll
.\jhdf.dll
.\jhdf.lib
.\jhdf5.dll
.\jhdf5.lib
.\libhdf.lib
.\libhdf5.lib
.\libjpeg.lib
.\libmfhdf.lib
.\libszip.lib
.\libxdr.lib
.\libzlib.lib
.\msvcp140.dll
.\tigl3.dll
.\tigl3.lib
.\tixi3.dll
.\vcomp140.dll
.\vcruntime140.dll


null

## Global scope imports

In [3]:
%import ncsa.hdf.hdf5lib.*
%import ncsa.hdf.object.*
%import ncsa.hdf.object.h5.*
%import ncsa.hdf.hdf5lib.structs.*
%import ncsa.hdf.hdf5lib.callbacks.*
%import java.util.ArrayList

## Explore a HDF5 file

In [4]:
package test.hdf5;

String fileName = "VeDSC_database.h5";
H5File fileH5 = null;
try {
    fileH5 = new H5File(fileName, FileFormat.READ);
    H5Group rootGroup = (H5Group) fileH5.get("/");
    String datasetFullName = "/KFv_vs_bv_over_dfv";
    
    int o_id = -1;
    int objType;
    H5O_info_t info;

    // check start from root
    o_id = rootGroup.open();
    info = H5.H5Oget_info(o_id);
    objType = info.type;

    class idata {
        public String link_name = null;
        public int link_type = -1;
        idata(String name, int type) {
            this.link_name = name;
            this.link_type = type;
        }
    }
    class H5O_iter_data implements H5O_iterate_t {
        public ArrayList<idata> iterdata = new ArrayList<idata>();
    }
    class H5O_iter_callback implements H5O_iterate_cb {
        public int callback(int group, String name, H5O_info_t info, H5O_iterate_t op_data) {
            idata id = new idata(name, info.type);
            ((H5O_iter_data)op_data).iterdata.add(id);

            System.out.print("/"); /* Print root group in object path */

            //Check if the current object is the root group, and if not print the full path name and type.

            if (name.charAt(0) == '.')         /* Root group, do not print '.' */
                System.out.println("  (Group)");
            else if(info.type == HDF5Constants.H5O_TYPE_GROUP ) 
                System.out.println(name + "  (Group)" );
            else if(info.type == HDF5Constants.H5O_TYPE_DATASET)
                System.out.println(name + "  (Dataset)");
            else if (info.type == HDF5Constants.H5O_TYPE_NAMED_DATATYPE )
                System.out.println(name + "  (Datatype)");
            else
                System.out.println(name + "  (Unknown)");

            return 0;
        }
    }
    
    H5O_iterate_t iter_data = new H5O_iter_data();
    H5O_iterate_cb iter_cb = new H5O_iter_callback();

    // recursive visit
    H5.H5Ovisit(
        fileH5.getFID(), 
        HDF5Constants.H5_INDEX_NAME, 
        HDF5Constants.H5_ITER_NATIVE, 
        iter_cb, 
        iter_data
    );

} catch (Exception e) {
    System.out.println("ERROR!");
    e.printStackTrace();
} finally {
    fileH5.close();
}

2018-02-05 14:50:55:882 +0100 [javash0] INFO ncsa.hdf.hdf5lib.H5 - HDF5 library: jhdf5
2018-02-05 14:50:55:885 +0100 [javash0] INFO ncsa.hdf.hdf5lib.H5 -  successfully loaded from java.library.path


/  (Group)
/KFv_vs_bv_over_dfv  (Group)
/KFv_vs_bv_over_dfv/data  (Dataset)
/KFv_vs_bv_over_dfv/var_0  (Dataset)
/KFv_vs_bv_over_dfv/var_1  (Dataset)
/KHf_vs_zh_over_bv1_high_wing  (Group)
/KHf_vs_zh_over_bv1_high_wing/data  (Dataset)
/KHf_vs_zh_over_bv1_high_wing/var_0  (Dataset)
/KHf_vs_zh_over_bv1_high_wing/var_1  (Dataset)
/KHf_vs_zh_over_bv1_high_wing/var_2  (Dataset)
/KHf_vs_zh_over_bv1_low_wing  (Group)
/KHf_vs_zh_over_bv1_low_wing/data  (Dataset)
/KHf_vs_zh_over_bv1_low_wing/var_0  (Dataset)
/KHf_vs_zh_over_bv1_low_wing/var_1  (Dataset)
/KHf_vs_zh_over_bv1_low_wing/var_2  (Dataset)
/KHf_vs_zh_over_bv1_mid_wing  (Group)
/KHf_vs_zh_over_bv1_mid_wing/data  (Dataset)
/KHf_vs_zh_over_bv1_mid_wing/var_0  (Dataset)
/KHf_vs_zh_over_bv1_mid_wing/var_1  (Dataset)
/KHf_vs_zh_over_bv1_mid_wing/var_2  (Dataset)
/KHv_vs_zh_over_bv1_high_wing  (Group)
/KHv_vs_zh_over_bv1_high_wing/data  (Dataset)
/KHv_vs_zh_over_bv1_high_wing/var_0  (Dataset)
/KHv_vs_zh_over_bv1_high_wing/var_1  (Dataset)
/KH

null